In [ ]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OPENAI_API_KEY from .env file
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ["OPENAI_ORGANIZATION"] = os.getenv('OPENAI_ORGANIZATION')

In [ ]:
from weaviate import Client
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda

# Initialize the Weaviate client
client = Client("http://localhost:8080")

def query_collection(query):
    response = (
        client.query
        .get("code_example", ["code"])
        .with_near_text({
            "concepts": [query]
        })
        .with_limit(4)
        .do()
    )
    return response

# Set up the system template with a variable for context
system_template = """
Generate working code for a Jupyter Notebook based on the user's request. Your code should use LangChain, and specifically use LangChain's Expression Language in structuring your code.

Strictly adhere to the code examples delimited by triple backticks below as context for how LangChain's API works. DO NOT use any patterns that you do not find in the example below, unless you are 100% certain they work in LangChain:

```
{context}
```

Before sharing, double check your work. I will tip you $100 if your code is perfect.

Do not explain your work, just share working code.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# Set up the human template with a variable for the request
human_template = """
{request}
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# Create the elements of the chain
setup_and_retrieval = RunnableParallel(
    {"context": RunnableLambda(query_collection), "request": RunnablePassthrough()}
)
prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
model = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
output_parser = StrOutputParser()

chain = setup_and_retrieval | prompt | model | output_parser

In [ ]:
request = """
# Create a chain that does the following:
# - Accept a string as input
# - Format message from System the input to send to OpenAI. The prompt has output instructions using Pydantic that expects a schema of an expert with name and description (in 10 words or less)
# - Pass the prompt to OpenAI
# - Parse the response using Pydantic
"""

# Stream the chain
for chunk in chain.stream(request):
    print(chunk, end="", flush=True)